In [1]:
import sys
sys.path.append('../../')
import glob
from pathlib import Path
import os

import pandas as pd
import numpy as np
import hashlib

from scripts.normalize_noaa_files import (
    unique_filenames_for_paths,
    unique_columns_for_paths,
    filename_index,
    format_filepaths_set
)
from config import OUTPUT_DIR, CLEAN_DATA_DIR
from scripts.shared_utils import (
    log_df
)
from scripts.normalize_data import (
    create_sample_name_noaa
)

In [2]:

base_dir = CLEAN_DATA_DIR



metadata_path = OUTPUT_DIR/'metadata'/'NOAA'/'noaa_dsdp_files.csv'


## Add eodp_id to data files

add hash eodp_id to each row in data files

In [14]:
metadata = pd.read_csv(metadata_path)
log_df(metadata)

(4477, 5)


,path,type,taxon_group,expedition,site
0,NOAA/DSDP_core_data/61/462/radiolar.csv,taxa,radiolarians,61,462
1,NOAA/DSDP_core_data/61/462/ageprof.csv,age,NaN,61,462
2,NOAA/DSDP_core_data/61/462/b_forams.csv,taxa,benthic_forams,61,462
3,NOAA/DSDP_core_data/61/462/p_forams.csv,taxa,planktic_forams,61,462
4,NOAA/DSDP_core_data/61/462/hr_desc.csv,hard_rock,NaN,61,462


In [10]:
metadata['type'].unique()

array(['taxa', 'age', 'hard_rock', 'lithology'], dtype=object)

In [14]:
metadata2 = metadata[metadata['type'].isin(['taxa','lithology'])]
len(metadata2)

3104

In [15]:

def add_eodp_id(file):
    def add_hash_id(id):
        return hashlib.md5(f'{file}{id}'.encode()).hexdigest()

    path = f"{CLEAN_DATA_DIR}/{file}"
    content = pd.read_csv(path, dtype=str)
    content.dropna(how='all', axis='rows', inplace=True)
       
    content['eodp_id'] = content.index 
    content['eodp_id'] = content['eodp_id'].apply(add_hash_id)
    
    content.to_csv(path, index=False)

res = [add_eodp_id(file) for file in metadata2['path']]


## add sample name

In [16]:
metadata = pd.read_csv(metadata_path)
log_df(metadata)

(4477, 5)


,path,type,taxon_group,expedition,site
0,NOAA/DSDP_core_data/61/462/radiolar.csv,taxa,radiolarians,61,462
1,NOAA/DSDP_core_data/61/462/ageprof.csv,age,NaN,61,462
2,NOAA/DSDP_core_data/61/462/b_forams.csv,taxa,benthic_forams,61,462
3,NOAA/DSDP_core_data/61/462/p_forams.csv,taxa,planktic_forams,61,462
4,NOAA/DSDP_core_data/61/462/hr_desc.csv,hard_rock,NaN,61,462


In [17]:
metadata2 = metadata[metadata['type'].isin(['taxa','lithology'])]
len(metadata2)

3104

In [1]:

def add_sample(file):
    path = f"{CLEAN_DATA_DIR}/{file}"
    content = pd.read_csv(path, dtype=str)
    create_sample_name_noaa(content)
           
    content.to_csv(path, index=False)

res = [add_sample(file) for file in metadata2['path']]
